# Getting Started with Pydantic AI for Creating Simple Agents  

[PydanticAI](https://github.com/pydantic/pydantic-ai) is a **Python agent framework** designed to simplify the process of building **production-grade applications** with Generative AI.  

## Key Features  

- 🚀 **Type-Safe**: Ensures powerful and informative type checking for better code reliability.  
- 🐍 **Python-Centric Design**: Uses familiar Python control flow and agent composition, making it easy to apply standard best practices.  
- 📏 **Structured Responses**: Leverages **Pydantic** for validating and structuring model outputs, ensuring consistency across runs.  

With **PydanticAI**, you can efficiently build AI-driven applications while maintaining **robust type safety, structured outputs, and Pythonic simplicity**.  


Step 1 install pydantic AI using  pip install pydantic-ai

In [1]:
%pip install pydantic-ai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creating a simple hello world of AI agents

In [7]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.agent import Agent

In [8]:
# It is interesting to note that there is no official class for Ollama model integration rather we have to use the OpenAIModel class for Ollama as well 😂
import nest_asyncio

nest_asyncio.apply()

Model_ID = 'llama3.2'  # Specify the model u want to run

Base_url = 'Http://localhost:11434/v1' # this is the default server url when running ollama

ollama_model = OpenAIModel(
    model_name= Model_ID,
    base_url= Base_url,
)


agent = Agent(
    model= ollama_model,
    system_prompt= ['make the reponse more informative by adding 1 line of infromation. ']        # you can change the system prompt to see the difference in result
)

import nest_asyncio

nest_asyncio.apply()      # These lines are to handle the event loops of pydantic and jupyter based enviornments ( add these in google colab as well, remove if running in simple .py file)

response = agent.run_sync('What is the captial of India ?')

print(response.data)
print(response.all_messages)
print(response)

The capital of India is New Delhi. New Delhi has been the seat of power for India since its independence in 1947 and serves as a hub for Indian politics, culture, and international diplomacy.
<bound method AgentRunResult.all_messages of AgentRunResult(data='The capital of India is New Delhi. New Delhi has been the seat of power for India since its independence in 1947 and serves as a hub for Indian politics, culture, and international diplomacy.')>
AgentRunResult(data='The capital of India is New Delhi. New Delhi has been the seat of power for India since its independence in 1947 and serves as a hub for Indian politics, culture, and international diplomacy.')


## Structured Outputs with PydanticAI  

Now that we have seen how to run a simple agent, let's explore how to ensure **consistent outputs** from LLMs using **structured responses**.  

### Why Structured Outputs?  

Generative AI models can produce highly variable responses. By leveraging **PydanticAI**, we can:  

- ✅ **Enforce Data Consistency**: Define strict output schemas to ensure reliable responses.  
- 🏗 **Improve Parsing & Validation**: Use Pydantic models to automatically validate and structure outputs.  
- 🚀 **Enhance Usability**: Make AI responses more predictable and easier to work with in production.  

Next, we’ll dive into how **PydanticAI** helps achieve structured outputs with minimal effort.  


In [10]:
from pydantic import BaseModel, Field  # we will specify our schema in by extending the basemodel class

class ResponseType(BaseModel):
    """Getting structured responses from LLMs along with metadata"""
    response: str  # We want the response type to be str
    sentiment: str = Field( description  = 'Customer sentiment analysis') # additional we want model to do


agent = Agent(
    model= ollama_model,
    result_type=ResponseType,
    system_prompt= ['You are customer support agent. Help as much as you can .Return results in Structured format only ']        # you can change the system prompt to see the difference in result
)
 

response = agent.run_sync("This product is very bad,quality is very bad, how to give review of this product")

# print(response)

print(response.data.model_dump_json(indent=2))

{
  "response": "Please provide a detailed description of your experience with the product, allowing us to assist you in writing a constructive review.",
  "sentiment": "neutral"
}



# Step-by-Step Guide: Structured Customer Support Agent with Pydantic and AI Agent

## Overview
This guide walks through the implementation of a structured AI-powered customer support agent using `pydantic_ai`. The system processes customer queries, extracts structured data, and provides intelligent responses.

---

## Step 1: Import Required Modules
First, import necessary modules for handling structured data and AI processing:

```python
from pydantic_ai import RunContext
from typing import Dict, List, Optional
```

- `RunContext` is used for managing structured responses.
- `List` and `Optional` help define flexible data structures.

---

## Step 2: Define Data Models
Define the schemas for `BookOrder` and `CustomerDetails` using `pydantic`.

### 2.1 Define the Book Order Schema
```python
class BookOrder(BaseModel):
    """Structure for order details."""
    order_id: str
    status: str
    items: List[str]
```
- `order_id`: Unique identifier for the order.
- `status`: Current status (e.g., `shipped`, `pending`).
- `items`: List of ordered books.

### 2.2 Define the Customer Schema
```python
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries."""
    customer_id: str
    name: str
    email: str
    orders: Optional[List[BookOrder]] = None
```
- `customer_id`: Unique customer identifier.
- `name`: Customer’s full name.
- `email`: Contact email.
- `orders`: A list of book orders (optional).

---

## Step 3: Create AI Agent with Dependencies
Define an AI agent using `pydantic_ai` and configure its response handling.

```python
agent5 = Agent(
    model=ollama_model,  # Define the AI model being used
    result_type=ResponseType,  # Expected response structure
    deps_type=CustomerDetails,  # The agent processes customer details
    retries=3,  # Number of retry attempts for failed queries
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Always greet the customer and provide a helpful response."
    ),
)
```
- Uses a predefined AI model (`ollama_model`).
- Accepts structured customer details (`CustomerDetails`).
- Retries up to three times for better robustness.
- A system prompt guides the AI behavior.

---

## Step 4: Modify System Prompt Dynamically
Update the system prompt dynamically to include customer details.

```python
@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {(ctx.deps)}"
```
- Enhances AI responses with real-time customer data.
- `ctx.deps` injects structured details into the prompt.

---

## Step 5: Create Customer Data
Instantiate a sample customer object with an order:

```python
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        BookOrder(order_id="12345", status="shipped", items=["Harry Potter and the Prisoner of Azkaban", "The Divine Comedy"]),
    ],
)
```
- A `CustomerDetails` object is created with one order.
- The order contains a status (`shipped`) and two book items.

---

## Step 6: Run the Agent with Customer Query
Use the agent to process a customer’s query about their orders.

```python
response = agent5.run_sync(user_prompt="What did I order?", deps=customer)
```
- The agent processes the structured customer details.
- The query "What did I order?" triggers the AI to extract order details.

---

## Step 7: Extract and Display the Response
Retrieve and print the structured response from the agent.

```python
response.all_messages()
print(response.data.model_dump_json(indent=2))
```
- `all_messages()` fetches AI-generated messages.
- `model_dump_json(indent=2)` neatly formats the response.

---

## Step 8: Print Customer and Response Details
Display structured details including the customer’s name, email, response, and order status.

```python
print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"{customer.orders[0].status}\n\n"
)
```
- Displays key customer details.
- Extracts the order status dynamically.

---

## Summary
This guide provides a structured approach to:
1. **Define schemas** using `pydantic` for structured data.
2. **Configure an AI-powered agent** for customer support.
3. **Integrate dynamic prompts** with customer details.
4. **Process and display structured responses** efficiently.

This ensures a seamless customer support experience with intelligent, structured responses.




In [11]:
from pydantic_ai import RunContext
from pydantic import BaseModel
from typing import Dict, List, Optional
#  Define order schema


class BookOrder(BaseModel):
    """Structure for order details."""

    order_id: str
    status: str
    items: List[str]


# Define customer schema
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries."""

    customer_id: str
    name: str
    email: str
    orders: Optional[List[BookOrder]] = None


# Agent with structured output and dependencies
agent5 = Agent(
    model=ollama_model,
    result_type=ResponseType,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
)


# Add dynamic system prompt based on dependencies
@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {(ctx.deps)}"  # These depend in some way on context that isn't known until runtime


customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        BookOrder(order_id="12345", status="shipped", items=["Harry Potter and the prisoner of Azakban", "The Divine Comedy"]),
    ],
)

response = agent5.run_sync(user_prompt="What did I order?", deps=customer)

response.all_messages()
print(response.data.model_dump_json(indent=2))

print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"{customer.orders[0].status}\n\n"
    
    
)


{
  "response": "You ordered Harry Potter and the prisoner of Azakban and The Divine Comedy.",
  "sentiment": ""
}
Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
You ordered Harry Potter and the prisoner of Azakban and The Divine Comedy.

Status:
shipped




Now Let's see to use Tools 

In [11]:
%pip install mysql-connector-python


   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.1 MB 8.5 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/16.1 MB 3.1 MB/s eta 0:00:05
   ------ --------------------------------- 2.6/16.1 MB 4.3 MB/s eta 0:00:04
   -------- ------------------------------- 3.4/16.1 MB 4.2 MB/s eta 0:00:04
   ----------- ---------------------------- 4.5/16.1 MB 4.1 MB/s eta 0:00:03
   ------------- -------------------------- 5.2/16.1 MB 4.1 MB/s eta 0:00:03
   -------------- ------------------------- 5.8/16.1 MB 4.1 MB/s eta 0:00:03
   ---------------- ----------------------- 6.8/16.1 MB 4.0 MB/s eta 0:00:03
   ------------------ --------------------- 7.6/16.1 MB 4.0 MB/s eta 0:00:03
   -------------------- ------------------- 8.4/16.1 MB 4.0 MB/s eta 0:00:02
   ---------------------- ----------------- 9.2/16.1 MB 4.0 MB/s eta 0:00:02
   ------------------------ --------------- 10.0/16.1 MB 4.0 MB/s eta 0:00:02
   --


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import mysql.connector

def create_database():
    conn = mysql.connector.connect(host="localhost", user="root", password="root")
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS shop_db")
    conn.close()

def create_table():
    conn = mysql.connector.connect(host="localhost", user="root", password="root", database="shop_db")
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS orders")  # Drop table if it exists
    cursor.execute("""
        CREATE TABLE orders (
            username VARCHAR(50),
            shipping_status VARCHAR(100),
            shipped_date DATE,
            delivery_status VARCHAR(100)
        )
    """)
    conn.commit()
    conn.close()


def insert_sample_data():
    conn = mysql.connector.connect(host="localhost", user="root", password="root", database="shop_db")
    cursor = conn.cursor()
    sample_data = [
        ("alice", "Shipped", "2024-12-01", "Delivered"),
        ("bob", "Out for delivery", "2024-12-02", "In transit"),
    
        ("charlie", None, None, None)
    ]
    cursor.executemany("INSERT INTO orders (username, shipping_status, shipped_date, delivery_status) VALUES (%s, %s, %s, %s)", sample_data)
    conn.commit()
    conn.close()

def get_order_details():
    while True:
        username = input("Enter username (or type 'exit' to quit): ").strip()
        if username.lower() == "exit":
            break  # Exit the loop

        conn = mysql.connector.connect(host="localhost", user="root", password="root", database="shop_db")
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM orders WHERE username = %s", (username,))
        orders = cursor.fetchall()
        conn.close()
        
        if not orders:
            print("No orders found for this user.")
        else:
            print("Order Details:")
            for order in orders:
                print(f"Username: {order[0]}, Shipping Status: {order[1] if order[1] else 'N/A'}, Shipped Date: {order[2] if order[2] else 'N/A'}, Delivery Status: {order[3] if order[3] else 'N/A'}")

if __name__ == "__main__":
    create_database()
    create_table()
    insert_sample_data()
    get_order_details()  # Now allows multiple queries and an exit option


if __name__ == "__main__":
    create_database()
    create_table()
    insert_sample_data()
    get_order_details()


Order Details:
Username: alice, Shipping Status: Shipped, Shipped Date: 2024-12-01, Delivery Status: Delivered
Order Details:
Username: alice, Shipping Status: Shipped, Shipped Date: 2024-12-01, Delivery Status: Delivered


In [12]:
from pydantic_ai import Tool

get_order_details()


def get_shipping_info(ctx: RunContext[CustomerDetails]) -> str:
    """Get the customer's shipping information."""
    return get_order_details[ctx.deps.orders[0].order_id]


# Agent with structured output and dependencies
agent5 = Agent(
    model=ollama_model,
    result_type=ResponseType,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
    tools=[Tool(get_shipping_info, takes_ctx=True)],  # Add tool via kwarg
)


@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {(ctx.deps)}"


response = agent5.run_sync(
    user_prompt="What's the status of my last order?", deps=customer
)

response.all_messages()
print(response.data.model_dump_json(indent=2))


Order Details:
Username: alice, Shipping Status: Shipped, Shipped Date: 2024-12-01, Delivery Status: Delivered
{
  "response": "The status of your last order is \"shipped\".",
  "sentiment": "neutral"
}
